In [27]:
from typing import List, Any, NamedTuple, Optional, Dict, TypeVar, Union
from collections import Counter, defaultdict
import numpy as np

### Entropy measure 

\begin{equation}
H(S) = -p_i * log_2p_i - ... -p_n * log_2p_n
\end{equation}
* $S$ is a set of instances 
* $p_i$ is a probability that instance in this set belongs to class $c_i$
* $n$ is the number of different classes

In [16]:
def entropy(class_proba: List[float]) -> float:
    '''
    Given list of class probabilities for a given set S,
    return the Entropy measure given by the equation:
    H(S) = -p_i * log2 pi - ... - p_n * log2 p_n.
    '''
    return sum([-p * np.log2(p) for p in class_proba if p > 0])

assert entropy([1.0]) == 0
assert entropy([0.5, 0.5]) == 1
assert 0.81 < entropy([0.25, 0.75]) < 0.82

In [32]:
def class_proba(labels: List[Any]) -> List[float]:
    '''
    Given list of labels in a set S, return probability that an instance
    of class c_i belongs to this set.
    [count(c_i)/total_count, ...]
    '''
    total_count = len(labels)
    return [count / total_count for count in Counter(labels).values()]

In [33]:
def data_entropy(labels: List[Any]) -> float:
    '''return entropy measure given list of labels'''
    return entropy(class_proba(labels))

assert data_entropy(['a']) == 0
assert data_entropy([True, False]) == 1
assert data_entropy([1, 2, 2, 2]) == entropy([0.25, 0.75])

### Entropy of a partition

\begin{equation}
H = q_1H(S_1) + ... + q_mH(S_m)
\end{equation}

* entropy of a partition is computed as a weighted sum of entropies of each of its subsets

In [34]:
def partition_entropy(subsets: List[List[Any]]) -> float:
    '''returns entrpy of a partition defined by the subset parameter'''
    
    total_count = sum(len(subset) for subset in subsets)
    return sum(data_entropy(subset) * len(subset) / total_count for subset in subsets)

In [35]:
class Candidate(NamedTuple):
    level: str
    lang: str
    tweets: bool
    phd: bool
    did_well: Optional[bool] = None # allowing unlabeled data

inputs = [
    Candidate('Senior', 'Java', False, False, False),
    Candidate('Senior', 'Java', False, True, False),
    Candidate('Mid', 'Python', False, False, True),
    Candidate('Junior', 'Python', False, False, True),
    Candidate('Junior', 'R', True, False, True),
    Candidate('Junior', 'R', True, True, False),
    Candidate('Mid', 'R', True, True, True),
    Candidate('Senior', 'Python', False, False, False),
    Candidate('Senior', 'R', True, False, True),
    Candidate('Junior', 'Python', True, False, True),
    Candidate('Senior', 'Python', True, True, True),
    Candidate('Mid', 'Python', False, True, True),
    Candidate('Mid', 'Java', True, False, True),
    Candidate('Junior', 'Python', False, True, False)
]

### ID3 Algorithm

* if all the data has the same label, create a leaf node that predicts the data and stop
* if the list of attributes is empty, create a leaf node that predicts the most common label 
and stop
* otherwise, try partitioning data by each of the attributes
* choose a partition with the lowest entropy
* add a decision noded based on the chosen attribute
* recurse on each partitioned subset using the remaining attributes

In [36]:
T = TypeVar('T') # generic type for inputs

def partition_by(inputs: List[T], attribute: str) -> Dict[Any, List[T]]:
    '''Partition inputs into lists based on the attribute parameter'''
    partitions: Dict[Any, List[T]] = defaultdict(list)
    
    for inp in inputs:
        key = getattr(inp, attribute) # value of a specified attribute
        partitions[key].append(inp)
        
    return partitions

In [37]:
def partition_entropy_by(inputs: List[Any], attribute: str, label_attribute: str) -> float:
    '''Compute entropy corresponding to the given partition'''
    
    partitions = partition_by(inputs, attribute)
    
    labels = [[getattr(inp, attribute) for inp in partition] for partition in partitions.values()]
    
    return partition_entropy(labels)

In [38]:
class Leaf(NamedTuple):
    value: Any
        
class Split(NamedTuple):
    attribute: str
    subtrees: dict
    default_value: Any = None
        
DecisionTree = Union[Leaf, Split]

In [39]:
def classify(tree: DecisionTree, inp: Any) -> Any:
    '''classify the input using the given decision tree'''
    
    if isinstance(tree, Leaf):
        return tree.value
    
    subtree_key = getattr(inp.attribute)
    if subtree_key not in tree.subtrees:
        return tree.default_value
    
    subtree = tree.subtrees[subtree_key]
    return classify(subtree, inp)

In [40]:
def build_tree_id3(inputs: List[Any], split_attributes: List[str], 
                   target_attribute: str) -> DecisionTree:
    # count target labels
    label_counts = Counter(getattr(inp, target_attribute) for inp in inputs)
    most_common_label = label_counts.most_common(1)[0][0]
    
    # if there is a unique label, predict it
    if len(label_counts) == 1:
        return Leaf(most_common_label)
    
    # if no split attributes are left, return the majority label
    if not split_attributes:
        return Leaf(most_common_label)
    
    # split by the best attribute
    def split_entropy(attribute: str) -> float:
        '''helper to find the best attribute'''
        return partition_entropy_by(inputs, attribute, target_attribute)
    
    best_attribute = min(split_attributes, key=split_entropy)
    
    partitions = partition_by(inputs, best_attribute)
    new_attributes = [a for a in split_attributes if a != best_attribute]
    
    # recursively build the subtree
    subtrees = {attribute_value: build_tree_id3(subset,
                                                new_attributes,
                                                target_attribute)
                for attribute_value, subset in partitions.items()}
    
    return Split(best_attribute, subtrees, default_value=most_common_label)

In [41]:
tree = build_tree_id3(inputs, ['level', 'lang', 'tweets', 'phd'], 'did_well')